1. Create a single dataframe with the concatenation of all input csv files, adding a column called country
2. Extract all videos that have no tag.
3. For each channel, determine the total number of views
4. Save all rows with disabled comments and disabled ratings, or that have video_error_or_removed in a new dataframe called excluded, and remove those rows from the original dataframe.
5. Add a like_ratio column storing the ratio between the number of likes and of dislikes
6. Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)
7. For each interval, determine the number of videos, average number of likes and of dislikes.
8. For each tag, determine the number of videos
Notice that tags contains a string with several tags.

9. Find the tags with the largest number of videos
10. For each (tag, country) pair, compute average ratio likes/dislikes
11. For each (trending_date, country) pair, the video with the largest number of views
12. Divide trending_date into three columns: year, month, day
13. For each (month, country) pair, the video with the largest number of views
14. Read all json files with the video categories
15. For each country, determine how many videos have a category that is not assignable.

In [36]:
import pandas as pd
import numpy as np
import glob
import json
from datetime import datetime
import os


### 1. Create a single dataframe with the concatenation of all input csv files


In [37]:
csv_files = glob.glob("trendingYT/*.csv")
csv_files 

['trendingYT\\CAvideos.csv',
 'trendingYT\\DEvideos.csv',
 'trendingYT\\FRvideos.csv',
 'trendingYT\\GBvideos.csv',
 'trendingYT\\INvideos.csv',
 'trendingYT\\JPvideos.csv',
 'trendingYT\\KRvideos.csv',
 'trendingYT\\MXvideos.csv',
 'trendingYT\\RUvideos.csv',
 'trendingYT\\USvideos.csv']

###  Adding a column called country and add all csv in single variable

In [38]:
dfs = []

for file in csv_files:
    # Extract file name without extension (CAvideos, USvideos, etc.)
    country = os.path.splitext(os.path.basename(file))[0]
    
    temp_df = pd.read_csv(file, encoding="latin1")
    temp_df["country"] = country
    
    dfs.append(temp_df)


### Concatenation of all input csv files

In [39]:
df = pd.concat(dfs, ignore_index=True)
df.head()


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃ© ...,CAvideos
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CAvideos
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,CAvideos
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CAvideos
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,ð§: https://ad.gt/yt-perfect\nð°: https://...,CAvideos


###  Remove "Videos" from country column

In [40]:

df['country'] = df['country'].str.replace('videos', '', case=False, regex=True)

df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃ© ...,CA
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,CA
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,ð§: https://ad.gt/yt-perfect\nð°: https://...,CA


### 2. Extract all videos that have no tag.

In [41]:
no_tags_df = df[df["tags"].isin(["[none]", "", np.nan])]
no_tags_df


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
41,JwboxqDylgg,17.14.11,Canada Soccer's Women's National Team v USA In...,Canada Soccer,17,2017-11-13T05:53:49.000Z,[none],36311,277,28,13,https://i.ytimg.com/vi/JwboxqDylgg/default.jpg,False,False,False,Canada Soccer's Women's National Team face riv...,CA
58,9B-q8h31Bpk,17.14.11,John Oliver Tackles Louis C.K. And Donald Trum...,TV Shows,22,2017-11-13T04:49:26.000Z,[none],106029,1270,101,181,https://i.ytimg.com/vi/9B-q8h31Bpk/default.jpg,False,False,False,"John Oliver on News, Politics ...",CA
78,1UE5Dq1rvUA,17.14.11,Taylor Swift Perform Ready For It - SNL,Ken Reactz,24,2017-11-12T05:18:02.000Z,[none],320964,8069,285,717,https://i.ytimg.com/vi/1UE5Dq1rvUA/default.jpg,False,False,False,Thanks for watching please subscribe and subsc...,CA
86,pmJQ4KwliX4,17.14.11,"LATEST Q POSTS: ROTHSCHILDS, HOUSE OF SAUD, lL...",James Munder,2,2017-11-12T21:25:40.000Z,[none],116820,1503,139,1066,https://i.ytimg.com/vi/pmJQ4KwliX4/default.jpg,False,False,False,https://pastebin.ca/3930472\n\nSupport My Chan...,CA
98,lHcXhBojpeQ,17.14.11,ä¸å±TVBè¦å¸ï¼ææ£10å¹´éæ¢ ç«¹é¦¬é«®å¦...,ææç¾æç,22,2017-11-12T12:49:50.000Z,[none],88061,47,58,17,https://i.ytimg.com/vi/lHcXhBojpeQ/default.jpg,False,False,False,NaN,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375815,VGykknw9eCM,18.14.06,"Made Defiant: The Mixtape ft. Neymar Jr., Kane...",Beats by Dre,10,2018-06-07T07:01:06.000Z,[none],3577614,12318,1345,1414,https://i.ytimg.com/vi/VGykknw9eCM/default.jpg,False,False,False,"When your time comes, you have two choices. Yo...",US
375819,fAIX12F6958,18.14.06,Bumblebee (2018) - Official Teaser Trailer - P...,Paramount Pictures,1,2018-06-05T07:00:01.000Z,[none],19864779,228670,16420,46318,https://i.ytimg.com/vi/fAIX12F6958/default.jpg,False,False,False,Every adventure has a beginning. Watch the off...,US
375865,gS1DbvHHVH0,18.14.06,Going in to brain surgery,Simone Giertz,28,2018-05-30T14:22:13.000Z,[none],1635301,120791,1098,20711,https://i.ytimg.com/vi/gS1DbvHHVH0/default.jpg,False,False,False,Weâll post an update on Instagram and Twitte...,US
375873,E4c7EE8_IX0,18.14.06,Weezer - Africa,weezer,10,2018-05-29T12:00:11.000Z,[none],4682557,115240,5069,9170,https://i.ytimg.com/vi/E4c7EE8_IX0/default.jpg,False,False,False,Stream + download the song: http://fanlink.to/...,US


### 3. For each channel, determine the total number of views

In [42]:
views_per_channel = (
    df.groupby("channel_title")["views"]
      .sum()
      .reset_index()
      .sort_values("views", ascending=False)
)

views_per_channel.head()


,channel_title,views
4564,ChildishGambinoVEVO,11016766510
15536,Marvel Entertainment,10430605449
17726,NickyJamTV,9479859505
18466,Ozuna,8623329509
28412,ibighit,8205572221


### 4. Save all rows with disabled comments and disabled ratings, or that have video_error_or_removed in a new dataframe called excluded, and remove those rows from the original dataframe.

In [43]:
excluded = df[
    (df["comments_disabled"] == True) |
    (df["ratings_disabled"] == True) |
    (df["video_error_or_removed"] == True)
]

excluded.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
67,amEZKmJQ4Io,17.14.11,Drako - Watch Me Do It [Official Video],babygranderecords,10,2017-10-23T19:38:36.000Z,"Drako|""Watch Me Do It""|""watch me""|""migos""|""dap...",25887,0,0,6,https://i.ytimg.com/vi/amEZKmJQ4Io/default.jpg,False,True,False,PURCHASE / STREAM WATCH ME DO IT https://fanli...,CA
70,TIvI07xLPnA,17.14.11,"The National for Sunday, November 12, 2017",The National,25,2017-11-13T03:06:10.000Z,"Canada|""CBC""|""CBC News""|""National""|""News""|""The...",13433,74,57,0,https://i.ytimg.com/vi/TIvI07xLPnA/default.jpg,True,False,False,"Welcome to The National, the flagship nightly ...",CA
82,Dgut-rlPVrk,17.14.11,Will Grace Davies make you love her? | Live Sh...,The X Factor UK,24,2017-11-12T19:59:14.000Z,"the x factor|""x factor""|""X factor UK""|""x facto...",261603,4276,2148,0,https://i.ytimg.com/vi/Dgut-rlPVrk/default.jpg,True,False,False,Visit the official site: http://itv.com/xfacto...,CA
173,H8IWLEtFt9A,17.14.11,IntÃ©grale - On n'est pas couchÃ© 11 novembre ...,On n'est pas couchÃ©,24,2017-11-12T01:44:07.000Z,"onpc|""on n'est pas couche""|""laurent ruquier""|""...",122282,0,0,0,https://i.ytimg.com/vi/H8IWLEtFt9A/default.jpg,True,True,False,IntÃ©grale - On n'est pas couchÃ© 11 novembre ...,CA
235,wDEA3rpYHnI,17.15.11,Marie-Louise Arsenault rÃ©plique Ã Denise Bom...,TV Classics,22,2017-11-13T01:26:37.000Z,Marie-Louise Arsenault qui rÃ©plique Ã Denise...,15800,88,0,0,https://i.ytimg.com/vi/wDEA3rpYHnI/default.jpg,True,False,False,Moment favori Ã la tÃ©lÃ© quÃ©bÃ©coise: Marie...,CA


###  Remove those rows from the original dataframe.

In [44]:
df = df.drop(excluded.index)


### 5. Add a like_ratio column storing the ratio between the number of likes and of dislikes

In [46]:
df["like_ratio"] = df["likes"] / (df["dislikes"] + 1)
df[["likes", "dislikes", "like_ratio"]].head()

,likes,dislikes,like_ratio
0,787425,43420,18.134658
1,127794,1688,75.662522
2,146035,5339,27.347378
3,132239,1989,66.451759
4,1634130,21082,77.509368


### 6. Cluster the publish time into 10-minute intervals (e.g. from 02:20 to 02:30)


##### Convert publish_time to Datetime

In [48]:
df["publish_time"] = pd.to_datetime(df["publish_time"], utc=True)
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10 17:00:03+00:00,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃ© ...,CA,18.134658
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13 17:00:00+00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA,75.662522
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,CA,27.347378
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA,66.451759
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09 11:04:14+00:00,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,ð§: https://ad.gt/yt-perfect\nð°: https://...,CA,77.509368


In [52]:
start_time = df["publish_time"]
end_time = start_time + pd.Timedelta(minutes=10)

df["publish_interval"] = (
    start_time.dt.strftime("%H:%M:%S") + "–" +
    end_time.dt.strftime("%H:%M:%S")
)

df[["publish_time", "publish_interval"]].head()


,publish_time,publish_interval
0,2017-11-10 17:00:03+00:00,17:00:03–17:10:03
1,2017-11-13 17:00:00+00:00,17:00:00–17:10:00
2,2017-11-12 19:05:24+00:00,19:05:24–19:15:24
3,2017-11-12 18:01:41+00:00,18:01:41–18:11:41
4,2017-11-09 11:04:14+00:00,11:04:14–11:14:14


### 7. For each interval, determine the number of videos, average number of likes and of dislikes.

In [53]:
interval_stats = (
    df.groupby("publish_interval")
      .agg(
          video_count=("video_id", "count"),
          avg_likes=("likes", "mean"),
          avg_dislikes=("dislikes", "mean")
      )
      .reset_index()
)

interval_stats.head()


,publish_interval,video_count,avg_likes,avg_dislikes
0,00:00:00–00:10:00,209,10253.038278,164.617225
1,00:00:01–00:10:01,346,36929.898844,632.751445
2,00:00:02–00:10:02,142,12188.774648,270.647887
3,00:00:03–00:10:03,195,618419.179487,28804.979487
4,00:00:04–00:10:04,109,10192.770642,572.559633


### 8. For each tag, determine the number of videos

In [58]:
df["tags_list"] = df["tags"].str.split("|")
df["tags_list"].head()

0    [Eminem, "Walk", "On", "Water", "Aftermath/Sha...
1    [plush, "bad unboxing", "unboxing", "fan mail"...
2    [racist superman, "rudy", "mancuso", "king", "...
3    [ryan, "higa", "higatv", "nigahiga", "i dare y...
4    [edsheeran, "ed sheeran", "acoustic", "live", ...
Name: tags_list, dtype: object

In [59]:
df_exploded = df.explode("tags_list")

In [66]:
tag_counts = df_exploded.groupby("tags_list")["video_id"].count().reset_index()
tag_counts.rename(columns={"video_id": "num_videos"}, inplace=True)
tag_counts = tag_counts.sort_values(by="num_videos")
tag_counts.head(20)  


,tags_list,num_videos
13,(BLAYA - FAZ GOSTOSO),1
15,"(ÑÐ°ÑÑÑ 5)""",1
868416,íì§ì¬ì ì¬ì¸ í¤,1
868420,íì,1
868422,í¬ë§ëíê¸°,1
386937,"""kebab fait maison""",1
386938,"""kebab frites""",1
386939,"""kebab maison""",1
386940,"""kebab maker""",1
386943,"""kebab sauce blanche harissa""",1


### 9. Find the tags with the largest number of videos

In [63]:
# Optional: remove '[none]' or empty tags
df_exploded = df_exploded[df_exploded["tags_list"] != "[none]"]
df_exploded = df_exploded[df_exploded["tags_list"] != ""]


In [67]:
top_tags = tag_counts.sort_values(by="num_videos", ascending=False)
top_tags.head(20)  # Top 20 most frequent tags


,tags_list,num_videos
336366,"""funny""",14834
277330,"""comedy""",11900
12344,"""2018""",10567
443876,"""news""",5653
436000,"""music""",5544
560940,"""video""",5338
11560,"""2017""",5334
363384,"""humor""",4992
535014,"""television""",4099
490026,"""review""",4006


### 10. For each (tag, country) pair, compute average ratio likes/dislikes

In [73]:
df["like_ratio"] = df.apply(lambda x: x["likes"] / x["dislikes"] if x["dislikes"] > 0 else x["likes"], axis=1)


In [75]:
tag_country_avg = (
    df_exploded
    .groupby(["tags_list", "country"])["like_ratio"]
    .mean()
    .reset_index()
    .rename(columns={"tags_list": "tag", "like_ratio": "avg_like_ratio"})
)

tag_country_avg.sort_values(by="avg_like_ratio", ascending=False).head(20)


,tag,country,avg_like_ratio
437528,"""fan shi""",GB,59188.670214
782128,"""wuyifan""",GB,59188.670214
281584,"""Wu Yi Fan""",GB,59188.670214
535484,"""kw""",GB,59188.670214
972633,"""å´äº¦å¡""",GB,59188.670214
549246,"""like that""",GB,59188.670214
549271,"""likethat""",GB,59188.670214
1058477,Kris Wu,GB,59188.670214
532747,"""kris""",GB,46951.141220
195661,"""Originals""",RU,5844.000000


### 11. For each (trending_date, country) pair, the video with the largest number of views

In [76]:
df["trending_date"] = pd.to_datetime(df["trending_date"], format="%y.%d.%m")


In [77]:
df_sorted = df.sort_values(["trending_date", "country", "views"], ascending=[True, True, False])


In [78]:
idx = df_sorted.groupby(["trending_date", "country"])["views"].idxmax()
top_videos = df_sorted.loc[idx].reset_index(drop=True)


In [79]:
top_videos[["trending_date", "country", "video_id", "title", "views"]].head(10)


,trending_date,country,video_id,title,views
0,2017-11-14,CA,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),33523622
1,2017-11-14,DE,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),33523622
2,2017-11-14,FR,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),33523622
3,2017-11-14,GB,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),33523622
4,2017-11-14,IN,ePO5M5DE01I,Tiger Zinda Hai | Official Trailer | Salman Kh...,35885754
5,2017-11-14,KR,n1WpP7iowLc,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,17158579
6,2017-11-14,MX,JzCsM1vtn78,THE LOGANG MADE HISTORY. LOL. AGAIN.,4477587
7,2017-11-14,RU,U_4BLnJuR3g,12 Ð»Ð°Ð¹ÑÑ Ð°ÐºÐ¾Ð² Ñ ÑÐµÑÐ¼Ð¾ÐºÐ»ÐµÐµÐ¼ ...,2109573
8,2017-11-14,US,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),33523622
9,2017-11-15,CA,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),39082222


### 12. Divide trending_date into three columns: year, month, day

In [80]:
df["trending_year"] = df["trending_date"].dt.year
df["trending_month"] = df["trending_date"].dt.month
df["trending_day"] = df["trending_date"].dt.day


In [81]:
df[["trending_date", "trending_year", "trending_month", "trending_day"]].head()


,trending_date,trending_year,trending_month,trending_day
0,2017-11-14,2017,11,14
1,2017-11-14,2017,11,14
2,2017-11-14,2017,11,14
3,2017-11-14,2017,11,14
4,2017-11-14,2017,11,14


### 13. For each (month, country) pair, the video with the largest number of views

In [82]:
df_sorted = df.sort_values(["trending_month", "country", "views"], ascending=[True, True, False])


In [83]:
# Get index of max views per (month, country)
idx = df_sorted.groupby(["trending_month", "country"])["views"].idxmax()

# Select the top videos
top_videos_month_country = df_sorted.loc[idx].reset_index(drop=True)


In [84]:
top_videos_month_country[["trending_month", "country", "video_id", "title", "views"]].head(10)


,trending_month,country,video_id,title,views
0,1,CA,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,43067983
1,1,DE,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,37728802
2,1,FR,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,37728802
3,1,GB,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,90598955
4,1,IN,dfnCAmr569k,"Taylor Swift - End Game ft. Ed Sheeran, Future",42019590
5,1,KR,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,37728802
6,1,MX,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,31680160
7,1,RU,dfnCAmr569k,"Taylor Swift - End Game ft. Ed Sheeran, Future",23198594
8,1,US,LsoLEjrDogU,Bruno Mars - Finesse (Remix) [Feat. Cardi B] [...,57951412
9,2,CA,xpVfcZ0ZcFM,Drake - Godâs Plan,47362934


### 14. Read all json files with the video categories


In [90]:
folder_path = r'D:\Data Science Milano Bicocca\First Year 2025-2026\Foundation of Computer Science\Project-2\trendingYT'

# Get all files that match *_category_id (or *_category_id.json if they have .json extension)
json_files = glob.glob(os.path.join(folder_path, "*_category_id*"))

json_files  # This should now list all your category files

['D:\\Data Science Milano Bicocca\\First Year 2025-2026\\Foundation of Computer Science\\Project-2\\trendingYT\\CA_category_id.json',
 'D:\\Data Science Milano Bicocca\\First Year 2025-2026\\Foundation of Computer Science\\Project-2\\trendingYT\\DE_category_id.json',
 'D:\\Data Science Milano Bicocca\\First Year 2025-2026\\Foundation of Computer Science\\Project-2\\trendingYT\\FR_category_id.json',
 'D:\\Data Science Milano Bicocca\\First Year 2025-2026\\Foundation of Computer Science\\Project-2\\trendingYT\\GB_category_id.json',
 'D:\\Data Science Milano Bicocca\\First Year 2025-2026\\Foundation of Computer Science\\Project-2\\trendingYT\\IN_category_id.json',
 'D:\\Data Science Milano Bicocca\\First Year 2025-2026\\Foundation of Computer Science\\Project-2\\trendingYT\\JP_category_id.json',
 'D:\\Data Science Milano Bicocca\\First Year 2025-2026\\Foundation of Computer Science\\Project-2\\trendingYT\\KR_category_id.json',
 'D:\\Data Science Milano Bicocca\\First Year 2025-2026\\Found

In [92]:
category_mappings = {}  # To store mapping per country

for file in json_files:
    country = os.path.basename(file).split("_")[0]  # Extract country code
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # Create mapping for this country
    mapping = {item["id"]: {"title": item["snippet"]["title"], 
                            "assignable": item["snippet"]["assignable"]}
               for item in data["items"]}
    
    category_mappings[country] = mapping


In [93]:
print(category_mappings)


{'CA': {'1': {'title': 'Film & Animation', 'assignable': True}, '2': {'title': 'Autos & Vehicles', 'assignable': True}, '10': {'title': 'Music', 'assignable': True}, '15': {'title': 'Pets & Animals', 'assignable': True}, '17': {'title': 'Sports', 'assignable': True}, '18': {'title': 'Short Movies', 'assignable': False}, '19': {'title': 'Travel & Events', 'assignable': True}, '20': {'title': 'Gaming', 'assignable': True}, '21': {'title': 'Videoblogging', 'assignable': False}, '22': {'title': 'People & Blogs', 'assignable': True}, '23': {'title': 'Comedy', 'assignable': True}, '24': {'title': 'Entertainment', 'assignable': True}, '25': {'title': 'News & Politics', 'assignable': True}, '26': {'title': 'Howto & Style', 'assignable': True}, '27': {'title': 'Education', 'assignable': True}, '28': {'title': 'Science & Technology', 'assignable': True}, '30': {'title': 'Movies', 'assignable': False}, '31': {'title': 'Anime/Animation', 'assignable': False}, '32': {'title': 'Action/Adventure', 'a

In [94]:
from pprint import pprint
pprint(category_mappings)


{'CA': {'1': {'assignable': True, 'title': 'Film & Animation'},
        '10': {'assignable': True, 'title': 'Music'},
        '15': {'assignable': True, 'title': 'Pets & Animals'},
        '17': {'assignable': True, 'title': 'Sports'},
        '18': {'assignable': False, 'title': 'Short Movies'},
        '19': {'assignable': True, 'title': 'Travel & Events'},
        '2': {'assignable': True, 'title': 'Autos & Vehicles'},
        '20': {'assignable': True, 'title': 'Gaming'},
        '21': {'assignable': False, 'title': 'Videoblogging'},
        '22': {'assignable': True, 'title': 'People & Blogs'},
        '23': {'assignable': True, 'title': 'Comedy'},
        '24': {'assignable': True, 'title': 'Entertainment'},
        '25': {'assignable': True, 'title': 'News & Politics'},
        '26': {'assignable': True, 'title': 'Howto & Style'},
        '27': {'assignable': True, 'title': 'Education'},
        '28': {'assignable': True, 'title': 'Science & Technology'},
        '30': {'assigna

### 15. For each country, determine how many videos have a category that is not assignable.

In [96]:
# Filter only non-assignable categories
df_non_assignable_categories = df_categories[df_categories["assignable"] == False]

# Create a dictionary per country for fast lookup
non_assignable_dict = df_non_assignable_categories.groupby("country")["category_id"].apply(set).to_dict()
non_assignable_dict


{'CA': {'18',
  '21',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44'},
 'DE': {'18',
  '21',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44'},
 'FR': {'18',
  '21',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44'},
 'GB': {'18',
  '21',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44'},
 'IN': {'18',
  '21',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44'},
 'JP': {'18',
  '21',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44'},
 'KR': {'18',
  '21',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44'},

In [102]:
# Function to check if a video category is non-assignable
def is_non_assignable(row):
    country = row['country']
    cat_id = str(row['category_id'])  # ensure string type matches JSON IDs
    return cat_id in non_assignable_dict.get(country, set())

# Create a new column in df
df['non_assignable_video'] = df.apply(is_non_assignable, axis=1)
df[['country', 'category_id','video_id', 'non_assignable_video',]].head()


,country,category_id,video_id,non_assignable_video
0,CA,10,n1WpP7iowLc,False
1,CA,23,0dBIkQ4Mz1M,False
2,CA,23,5qpjK5DgCt4,False
3,CA,24,d380meD0W0M,False
4,CA,10,2Vv-BfVoq4g,False


In [98]:
non_assignable_videos_count = df.groupby('country')['non_assignable_video'].sum().reset_index()
non_assignable_videos_count.rename(columns={'non_assignable_video': 'non_assignable_videos_count'}, inplace=True)
non_assignable_videos_count


,country,non_assignable_videos_count
0,CA,130
1,DE,102
2,FR,110
3,GB,20
4,IN,220
5,JP,0
6,KR,167
7,MX,3
8,RU,183
9,US,57
